This notebook may take X minutes to run.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
#import matplotlib.pyplot as plt
import gzip
import json
#from tabulate import tabulate
import bz2
#import os
from tqdm import tqdm

# Load the Data

## Helper functions and variables

In [10]:
def delta_date(x):
    return (str_to_date(x['end_date']) - str_to_date(x['start_date'])).days + 1

In [11]:
def str_to_date(d):
    return datetime.strptime(d, "%Y-%m-%d")

In [30]:
datapath="irrs/"

In [399]:
def overlap_db(db):
    timeless_overlap = db.merge(bgp, left_on='route', right_on='prefix', suffixes=('_route', '_prefix'))
    timeless_overlap['end_date'] = pd.to_datetime(timeless_overlap['end_date'], format="%Y-%m-%d")
    timeless_overlap['start_date'] = pd.to_datetime(timeless_overlap['start_date'], format="%Y-%m-%d")
    timeless_overlap['start']=pd.to_datetime(timeless_overlap['start'], unit='s').dt.strftime('%Y-%m-%d')
    timeless_overlap['end']=pd.to_datetime(timeless_overlap['end'], unit='s').dt.strftime('%Y-%m-%d')
    #timeless_overlap['origin']=timeless_overlap['origin'].str.replace('AS',"")
    actual_overlap = timeless_overlap[(timeless_overlap.end_date > timeless_overlap.start)&(timeless_overlap.start_date < timeless_overlap.end)]
    actual_overlap['asn']='AS'+actual_overlap['asn'].astype(str)
    return actual_overlap

## Load BGP

In [83]:
bgp = pd.read_csv('pfx2as_2021_2023_merged.csv.gz', names=['prefix', 'asn', 'start', 'end'], delimiter=' ',low_memory=False)
bgp['duration'] = bgp['end'] - bgp['start']
bgpagg = bgp.groupby(['prefix', 'asn']).agg({'duration':'sum'}).reset_index()
bgpagg['asn']='AS'+bgpagg['asn'].astype(str)

## Load Altdb

In [342]:
altdb = pd.read_json(datapath+'altdb/altdb.route.json.gz', lines=True)
altdb['lifetime'] = altdb.apply(delta_date, axis=1)
altdb_overlap=overlap_db(altdb)
altdbagg = altdb_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

<ipython-input-341-bca60fb6450b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_overlap['asn']='AS'+actual_overlap['asn'].astype(str)


## Load Radb

In [400]:
radb = pd.read_json(datapath+'radb/radb.route.json.gz', lines=True)
radb['lifetime'] = radb.apply(delta_date, axis=1)
radb_overlap=overlap_db(radb)
radbagg = radb_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

<ipython-input-399-bca60fb6450b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_overlap['asn']='AS'+actual_overlap['asn'].astype(str)


## Load Afrinic

In [288]:
afrinic = pd.read_json(datapath+'afrinic/afrinic.route.json.gz', lines=True)
afrinic['lifetime'] = afrinic.apply(delta_date, axis=1)
afrinic_overlap=overlap_db(afrinic)
afrinicagg = afrinic_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Jpirr

In [289]:
jpirr = pd.read_json(datapath+'jpirr/jpirr.route.json.gz', lines=True)
jpirr['lifetime'] = jpirr.apply(delta_date, axis=1)
jpirr_overlap=overlap_db(jpirr)
jpirragg = jpirr_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Canarie

In [290]:
canarie = pd.read_json(datapath+'canarie/canarie.route.json.gz', lines=True)
canarie['lifetime'] = canarie.apply(delta_date, axis=1)
canarie_overlap=overlap_db(canarie)
canarieagg = canarie_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Apnic

In [291]:
apnic = pd.read_json(datapath+'apnic/apnic.route.json.gz', lines=True)
apnic['lifetime'] = apnic.apply(delta_date, axis=1)
apnic_overlap=overlap_db(apnic)
apnicagg = apnic_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Arin-Nonauth

In [292]:
arin_nonauth = pd.read_json(datapath+'arin-nonauth/arin-nonauth.route.json.gz', lines=True)
arin_nonauth['lifetime'] = arin_nonauth.apply(delta_date, axis=1)
arin_nonauth_overlap=overlap_db(arin_nonauth)
arin_nonauthagg = arin_nonauth_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Level3

In [293]:
level3 = pd.read_json(datapath+'level3/level3.route.json.gz', lines=True)
level3['lifetime'] = level3.apply(delta_date, axis=1)
level3_overlap=overlap_db(level3)
level3agg = level3_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Nestegg

In [294]:
nestegg = pd.read_json(datapath+'nestegg/nestegg.route.json.gz', lines=True)
nestegg['lifetime'] = nestegg.apply(delta_date, axis=1)
nestegg_overlap=overlap_db(nestegg)
nesteggagg = nestegg_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load BBoi

In [295]:
bboi= pd.read_json(datapath+'bboi/bboi.route.json.gz', lines=True)
bboi['lifetime'] = bboi.apply(delta_date, axis=1)
bboi_overlap=overlap_db(bboi)
bboiagg = bboi_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load idnic

In [296]:
idnic= pd.read_json(datapath+'idnic/idnic.route.json.gz', lines=True)
idnic['lifetime'] = idnic.apply(delta_date, axis=1)
idnic_overlap=overlap_db(idnic)
idnicagg = idnic_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load wcgdb

In [297]:
wcgdb= pd.read_json(datapath+'wcgdb/wcgdb.route.json.gz', lines=True)
wcgdb['lifetime'] = wcgdb.apply(delta_date, axis=1)
wcgdb_overlap=overlap_db(wcgdb)
wcgdbagg = wcgdb_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load RGNET

In [298]:
rgnet= pd.read_json(datapath+'rgnet/rgnet.route.json.gz', lines=True)
rgnet['lifetime'] = rgnet.apply(delta_date, axis=1)
rgnet_overlap=overlap_db(rgnet)
rgnetagg = rgnet_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load tc

In [299]:
tc= pd.read_json(datapath+'tc/tc.route.json.gz', lines=True)
tc['lifetime'] = tc.apply(delta_date, axis=1)
tc_overlap=overlap_db(tc)
tcagg = tc_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Lacnic

In [300]:
lacnic= pd.read_json(datapath+'lacnic/lacnic.route.json.gz', lines=True)
lacnic['lifetime'] = lacnic.apply(delta_date, axis=1)
lacnic_overlap=overlap_db(lacnic)
lacnicagg = lacnic_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Ripe-nonauth

In [301]:
ripe_nonauth= pd.read_json(datapath+'ripe-nonauth/ripe-nonauth.route.json.gz', lines=True)
ripe_nonauth['lifetime'] = ripe_nonauth.apply(delta_date, axis=1)
ripe_nonauth_overlap=overlap_db(ripe_nonauth)
ripe_nonauthagg = ripe_nonauth_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Openface

In [302]:
openface= pd.read_json(datapath+'openface/openface.route.json.gz', lines=True)
openface['lifetime'] = openface.apply(delta_date, axis=1)
openface_overlap=overlap_db(openface)
openfaceagg = openface_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Panix

In [303]:
panix= pd.read_json(datapath+'panix/panix.route.json.gz', lines=True)
panix['lifetime'] = panix.apply(delta_date, axis=1)
panix_overlap=overlap_db(panix)
panixagg = panix_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Arin

In [304]:
arin= pd.read_json(datapath+'arin/arin.route.json.gz', lines=True)
arin['lifetime'] = arin.apply(delta_date, axis=1)
arin_overlap=overlap_db(arin)
arinagg = arin_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Ripe

In [305]:
ripe= pd.read_json(datapath+'ripe/ripe.route.json.gz', lines=True)
ripe['lifetime'] = ripe.apply(delta_date, axis=1)
ripe_overlap=overlap_db(ripe)
ripeagg = ripe_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

## Load Nttcom

In [306]:
nttcom= pd.read_json(datapath+'nttcom/nttcom.route.json.gz', lines=True)
nttcom['lifetime'] = nttcom.apply(delta_date, axis=1)
nttcom_overlap=overlap_db(nttcom)
nttcomagg = nttcom_overlap.groupby(['route', 'origin']).agg({'lifetime':'sum'}).reset_index()

# Table 2: Calculate number of prefix-origin pairs in every IRR, overlap of prefix-origin pairs with BGP and all IRRs, & generate table

## Helper functions

In [307]:
def numerator_overlap(dfagg):
    overlap=len(dfagg.merge(aggbgp, left_on=['route', 'origin'], right_on=['prefix', 'asn']))
    return overlap

In [308]:
def denominator(df):
    return len(df)

In [313]:
def percentage_frac(dfagg,df):
    num=numerator_overlap(dfagg)
    denom=denominator(df)
    return str(round((num*100/denom),2))+"\% ("+str(num)+"/"+str(denom)+")"

In [314]:
print("\\begin{table}[!h]")
print("\\begin{tabular}{l|l|l}")
print("\\toprule")
print("IRR & \# Prefix-Origin Pairs & \% Prefix-Origin Pairs in BGP "+chr(92)+chr(92))
print("\\midrule")
print("AFRINIC","&",denominator(afrinic),"&", percentage_frac(afrinicagg,afrinic), chr(92)+chr(92))
print("\\midrule")
print("ALTDB","&",denominator(altdb),"&", percentage_frac(altdbagg,altdb), chr(92)+chr(92))
print("\\midrule")
print("APNIC","&",denominator(apnic),"&", percentage_frac(apnicagg,apnic), chr(92)+chr(92))
print("\\midrule")
print("ARIN","&",denominator(arin),"&", percentage_frac(arinagg,arin), chr(92)+chr(92))
print("\\midrule")
print("ARIN-NA","&",denominator(arin_nonauth),"&", percentage_frac(arin_nonauthagg,arin_nonauth), chr(92)+chr(92))
print("\\midrule")
print("BBOI","&",denominator(bboi),"&", percentage_frac(bboiagg,bboi), chr(92)+chr(92))
print("\\midrule")
print("CANARIE","&",denominator(canarie),"&", percentage_frac(canarieagg,canarie), chr(92)+chr(92))
print("\\midrule")
print("IDNIC","&",denominator(idnic),"&", percentage_frac(idnicagg,idnic), chr(92)+chr(92))
print("\\midrule")
print("JPIRR","&",denominator(jpirr),"&", percentage_frac(jpirragg,jpirr), chr(92)+chr(92))
print("\\midrule")
print("LACNIC","&",denominator(lacnic),"&", percentage_frac(lacnicagg,lacnic), chr(92)+chr(92))
print("\\midrule")
print("LEVEL3","&",denominator(level3),"&", percentage_frac(level3agg,level3), chr(92)+chr(92))
print("\\midrule")
print("NESTEGG","&",denominator(nestegg),"&", percentage_frac(nesteggagg,nestegg), chr(92)+chr(92))
print("\\midrule")
print("NTTCOM","&",denominator(nttcom),"&", percentage_frac(nttcomagg,nttcom), chr(92)+chr(92))
print("\\midrule")
print("OPENFACE","&",denominator(openface),"&", percentage_frac(openfaceagg,openface), chr(92)+chr(92))
print("\\midrule")
print("PANIX","&",denominator(panix),"&", percentage_frac(panixagg,panix), chr(92)+chr(92))
print("\\midrule")
print("RADB","&",denominator(radb),"&", percentage_frac(radbagg,radb), chr(92)+chr(92))
print("\\midrule")
print("RGNET","&",denominator(rgnet),"&", percentage_frac(rgnetagg,rgnet), chr(92)+chr(92))
print("\\midrule")
print("RIPE","&",denominator(ripe),"&", percentage_frac(ripeagg,ripe), chr(92)+chr(92))
print("\\midrule")
print("RIPE-NA","&",denominator(ripe_nonauth),"&", percentage_frac(ripe_nonauthagg,ripe_nonauth), chr(92)+chr(92))
print("\\midrule")
print("TC","&",denominator(tc),"&", percentage_frac(tcagg,tc), chr(92)+chr(92))
print("\\midrule")
print("WCGDB","&",denominator(wcgdb),"&", percentage_frac(wcgdbagg,wcgdb), chr(92)+chr(92))
print("\\midrule")
print("\\bottomrule")
print("\\end{tabular}")
print("\\caption{insert}")
print("\\end{table}")

\begin{table}[!h]
\begin{tabular}{l|l|l}
\toprule
IRR & \# Prefix-Origin Pairs & \% Prefix-Origin Pairs in BGP \\
\midrule
AFRINIC & 106842 & 20.37\% (21759/106842) \\
\midrule
ALTDB & 27493 & 58.35\% (16043/27493) \\
\midrule
APNIC & 701196 & 17.32\% (121480/701196) \\
\midrule
ARIN & 104298 & 49.35\% (51467/104298) \\
\midrule
ARIN-NA & 66126 & 18.4\% (12169/66126) \\
\midrule
BBOI & 952 & 51.89\% (494/952) \\
\midrule
CANARIE & 1460 & 58.42\% (853/1460) \\
\midrule
IDNIC & 6158 & 64.44\% (3968/6158) \\
\midrule
JPIRR & 42546 & 21.42\% (9114/42546) \\
\midrule
LACNIC & 97908 & 10.01\% (9800/97908) \\
\midrule
LEVEL3 & 121534 & 18.47\% (22452/121534) \\
\midrule
NESTEGG & 4 & 75.0\% (3/4) \\
\midrule
NTTCOM & 502051 & 13.77\% (69146/502051) \\
\midrule
OPENFACE & 17 & 41.18\% (7/17) \\
\midrule
PANIX & 40 & 15.0\% (6/40) \\
\midrule
RADB & 1875251 & 23.7\% (444479/1875251) \\
\midrule
RGNET & 44 & 47.73\% (21/44) \\
\midrule
RIPE & 469651 & 56.43\% (265001/469651) \\
\midrule
RIPE-NA 

# Section 7: Irregular Route Objects

## Helper functions

### as2org

In [475]:
orgs=pd.read_csv('as2org_clean.csv')
orgs['aut']=orgs['aut'].map(str)
orgs_ases_gt=list(orgs['aut'].unique())

### AS relationships

In [718]:
with bz2.open('20230401.as-rel.txt.bz2','rt') as f:
    content=f.read()
lines=content.split('\n')
clean_lines=[]
ases=[]
mypeers={}
mycustomers={}
myproviders={}
#the above dictionaries are organized s.t. mypeers[AS1]=AS2 and mypeers[AS2]=AS1
for l in tqdm(lines):
    if not l.startswith('#'):
        clean_lines.append(l)
        if len(l.split('|'))==3:
            if l.split('|')[2]=='0':
                if l.split('|')[0] in mypeers:
                    mypeers[l.split('|')[0]].append(l.split('|')[1])
                else:
                    mypeers[l.split('|')[0]]=[(l.split('|')[1])]
                if l.split('|')[1] in mypeers:
                    mypeers[l.split('|')[1]].append(l.split('|')[0])
                else:
                    mypeers[l.split('|')[1]]=[(l.split('|')[0])]
            if l.split('|')[2]=='-1': #AS1 is a provider of AS2, AS2 is a customer of AS1 AS1|AS2|-1
                if l.split('|')[1] in myproviders:
                    myproviders[l.split('|')[1]].append(l.split('|')[0])
                else:
                    myproviders[l.split('|')[1]]=[(l.split('|')[0])]
                if l.split('|')[0] in mycustomers:
                    mycustomers[l.split('|')[0]].append(l.split('|')[1])
                else:
                    mycustomers[l.split('|')[0]]=[(l.split('|')[1])]

## Section 7.1 and Table 3: RADB Analysis

### Import inconsistent prefixes

In [385]:
radb_cover=pd.read_csv('radb_cover.csv')

In [386]:
radb_inconsistent=set(list(radb_cover.nonauth.unique()))

In [401]:
len(radb_inconsistent)

150402

### Appear in BGP and inconsistent

In [402]:
bgp_inconsistent_radb=radb_overlap[radb_overlap['prefix'].isin(radb_inconsistent)]

In [403]:
print(bgp_inconsistent_radb.prefix.nunique(),"appear in BGP and are inconsistent.")

56803 appear in BGP and are inconsistent.


In [872]:
print(round(bgp_inconsistent_radb.prefix.nunique()*100/len(radb_inconsistent),2),"%")

37.77 %


### Full overlap

In [404]:
in_b_r=bgp_inconsistent_radb.prefix.unique()

In [405]:
df_in_b_r=radb_overlap[radb_overlap['route'].isin(in_b_r)]

In [406]:
route_r_b=df_in_b_r.groupby('route').agg({'asn':set,'origin':set}).reset_index()

In [659]:
route_r_b['related']=False #this is True when the ASes are the same or related

In [660]:
route_r_b['Fullmatch']=(route_r_b['asn']==route_r_b['origin']) #python discards order when evaluating set equality

In [876]:
full_overlap=route_r_b[route_r_b['Fullmatch']]['route'].nunique()

In [877]:
print(full_overlap,"have a full overlap.")

3002 have a full overlap.


In [662]:
notfullmatch=route_r_b[~route_r_b['Fullmatch']]
print(notfullmatch['route'].nunique()," do not have a full overlap.")

53801  do not have a full overlap.


In [410]:
53801+3002

56803

In [878]:
print(round(full_overlap*100/bgp_inconsistent_radb.prefix.nunique(),2),"%")

5.28 %


### Partial Overlap

In [838]:
route_r_b['intersec'] = route_r_b.apply(lambda x: set(x['asn']).intersection(set(x['origin'])), axis=1)
route_r_b['partial_match']=(route_r_b['intersec']!=set())
route_r_b['all_ases']=route_r_b.apply(lambda x: (x['asn'] | x['origin']), axis=1)

In [839]:
partial=route_r_b[(~route_r_b['Fullmatch']) & (route_r_b['partial_match'])]

In [840]:
print(partial['route'].nunique(),"prefixes have a partial overlap.")

32838 prefixes have a partial overlap.


#### Do they belong to same org?

In [841]:
pa_mis_rs=partial['route'].unique() # checking that the AS partial mismatches might be from the same org
r2orgspa={}
ctr=0
tot=len(pa_mis_rs)
for r in tqdm(pa_mis_rs):
    r2orgspa[r]=[]
    ases_to_check=list(partial[partial['route']==str(r)]['all_ases'])[0]
    clean=[i.replace("'","") for i in ases_to_check]
    clean=[i.replace("AS","") for i in ases_to_check]
    for a in clean:
        if a in orgs_ases_gt:
            check_org=list(orgs[orgs['aut']==a]['org_id'].unique())[0]
            r2orgspa[r].append(check_org)
        if a not in orgs_ases_gt:
            r2orgspa[r].append('missing')
for r in r2orgspa:
    r2orgspa[r]=set(r2orgspa[r])

100%|██████████| 32838/32838 [27:36<00:00, 19.82it/s] 


In [842]:
as_mismatch_same_org=[]
for r in r2orgspa:
    if (len(r2orgspa[r])==1) and (r2orgspa[r]!='missing'):
        as_mismatch_same_org.append(r)

In [881]:
full_overlap+=len(as_mismatch_same_org)

In [843]:
print(len(as_mismatch_same_org),"prefixes belong to the same org.")

90 prefixes belong to the same org.


In [851]:
#route_r_b['same_org']=False
partial['same_org']=False
for i in list(route_r_b.index[route_r_b['route'].isin(as_mismatch_same_org)]):
    #route_r_b.at[i,'related']=True
    #route_r_b.at[i,'same_org']=True
    partial.at[i,'related']=True
    partial.at[i,'same_org']=True

In [854]:
len(partial[partial['same_org']]+partial[~partial['related']])

32838

#### If not, is there a customer or peer relationship?

In [864]:
check_rel=partial[~partial['related']]['route'].unique()
related=[]
unrelated=[]
for r in tqdm(check_rel):
    check_ases=list(list(route_r_b[route_r_b['route']==str(r)]['all_ases'])[0])
    #use first AS to check relationships
    token_as=check_ases[0].lower().split('as')[1]
    #check if the rest of the ases are peers, customers, or providers
    token_peers=[]
    token_providers=[]
    token_customers=[]
    if token_as in myproviders:
        token_providers=myproviders[token_as]
    if token_as in mycustomers:
        token_customers=mycustomers[token_as]
    if token_as in mypeers:
        token_peers=mypeers[token_as]
    for a in check_ases:
        clean=a.lower().split('as')[1]
        if clean==token_as:
            continue
        if clean in token_peers:
            related.append(r)
        if clean in token_customers:
            related.append(r)
        if clean in token_providers:
            related.append(r)
    if r not in related:
        unrelated.append(r)
related=set(related)
unrelated=set(unrelated)

100%|██████████| 32748/32748 [04:58<00:00, 109.60it/s]


In [867]:
len(related)+len(unrelated)

32748

In [868]:
for i in list(route_r_b.index[route_r_b['route'].isin(related)]):
    route_r_b.at[i,'related']=True

In [882]:
full_overlap+=len(related)

In [869]:
print(len(related),"routes have a customer or provider or peer relationship.")

9648 routes have a customer or provider or peer relationship.


In [870]:
print(len(unrelated),"routes do not have a customer or provider or peer relationship.")

23100 routes do not have a customer or provider or peer relationship.


### No Overlap

In [887]:
print(route_r_b[~route_r_b['partial_match']]['route'].nunique(),"total mismatches.")

20963 total mismatches.


In [892]:
total=route_r_b[(~route_r_b['partial_match'])&(~route_r_b['related'])]

In [893]:
tot_mis_rs=total['route'].unique() # checking that the AS mismatches might be from the same org
r2orgs={}
ctr=0
tot=len(tot_mis_rs)
for r in tqdm(tot_mis_rs):
    r2orgs[r]=[]
    ases_to_check=list(total[total['route']==str(r)]['all_ases'])[0]
    clean=[i.replace("'","") for i in ases_to_check]
    clean=[i.replace("AS","") for i in ases_to_check]
    for a in clean:
        if a in orgs_ases_gt:
            check_org=list(orgs[orgs['aut']==a]['org_id'].unique())[0]
            r2orgs[r].append(check_org)
        if a not in orgs_ases_gt:
            r2orgs[r].append('missing')
for r in r2orgs:
    r2orgs[r]=set(r2orgs[r])

100%|██████████| 20963/20963 [17:18<00:00, 20.18it/s]


In [894]:
as_mismatch_same_org=[]
for r in r2orgs:
    if (len(r2orgs[r])==1) and (r2orgs[r]!='missing'):
        as_mismatch_same_org.append(r)

In [895]:
print(len(as_mismatch_same_org),"prefixes belong to the same org.")

52 prefixes belong to the same org.


In [896]:
print(len(tot_mis_rs)-len(as_mismatch_same_org),"prefixes do not belong to the same org.")

20911 prefixes do not belong to the same org.


In [897]:
route_r_b['same_org']=False
total['same_org']=False
for i in list(route_r_b.index[route_r_b['route'].isin(as_mismatch_same_org)]):
    route_r_b.at[i,'same_org']=True
    route_r_b.at[i,'related']=True
    total.at[i,'same_org']=True
    total.at[i,'related']=True

In [898]:
total['same_org']=False
for i in list(route_r_b.index[route_r_b['route'].isin(as_mismatch_same_org)]):
    total.at[i,'same_org']=True
    total.at[i,'related']=True

<ipython-input-898-6e4c18a6ac7d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total['same_org']=False


#### If not, is there a customer or peer relationship?

In [901]:
check_rel=total[~total['related']]['route'].unique()
related=[]
unrelated=[]
for r in tqdm(check_rel):
    check_ases=list(list(route_r_b[route_r_b['route']==str(r)]['all_ases'])[0])
    #use first AS to check relationships
    token_as=check_ases[0].lower().split('as')[1]
    #check if the rest of the ases are peers, customers, or providers
    token_peers=[]
    token_providers=[]
    token_customers=[]
    if token_as in myproviders:
        token_providers=myproviders[token_as]
    if token_as in mycustomers:
        token_customers=mycustomers[token_as]
    if token_as in mypeers:
        token_peers=mypeers[token_as]
    for a in check_ases:
        clean=a.lower().split('as')[1]
        if clean==token_as:
            continue
        if clean in token_peers:
            related.append(r)
        if clean in token_customers:
            related.append(r)
        if clean in token_providers:
            related.append(r)
    if r not in related:
        unrelated.append(r)
related=set(related)
unrelated=set(unrelated)

  6%|▋         | 1320/20911 [00:10<02:42, 120.59it/s]

In [904]:
print(len(related)+len(unrelated))

In [820]:
for i in list(route_r_b.index[route_r_b['route'].isin(related)]):
    route_r_b.at[i,'related']=True

In [822]:
print(len(related),"routes have a customer or provider or peer relationship.")

16 routes have a customer or provider or peer relationship.


In [823]:
print(len(unrelated),"routes do not have a customer or provider or peer relationship.")

295 routes do not have a customer or provider or peer relationship.


## Section 7.2: ALTDB Analysis

### Import inconsistent prefixes

In [627]:
altdb_cover=pd.read_csv('altdb_cover.csv')

In [628]:
altdb_inconsistent=list(altdb_cover.nonauth.unique())

In [629]:
len(altdb_inconsistent)

1206

In [630]:
altdb[altdb['route'].isin(altdb_inconsistent)]['route'].nunique()

1206

### Appear in BGP and inconsistent

In [631]:
bgp_inconsistent_altdb=altdb_overlap[altdb_overlap['prefix'].isin(altdb_inconsistent)]
#using altdb_overlap to get overlapping time for both altdb and bgp

In [632]:
print(bgp_inconsistent_altdb.prefix.nunique(),"appear in BGP and are inconsistent.")

619 appear in BGP and are inconsistent.


### Full overlap

In [633]:
in_b_a=bgp_inconsistent_altdb.prefix.unique()

In [634]:
df_in_b_a=altdb_overlap[altdb_overlap['route'].isin(in_b_a)]

In [663]:
route_a_b['related']=False #this is True when the ASes are the same or related

In [635]:
route_a_b=df_in_b_a.groupby('route').agg({'asn':set,'origin':set}).reset_index()

In [664]:
route_a_b['Fullmatch']=(route_a_b['asn']==route_a_b['origin']) #python discards order when evaluating set equality

In [667]:
fully_matched=route_a_b[route_a_b['Fullmatch']]['route'].unique()

In [668]:
for i in list(route_a_b.index[route_a_b['route'].isin(fully_matched)]):
    route_a_b.at[i,'related']=True

In [669]:
print(route_a_b[route_a_b['related']]['route'].nunique()," have a full overlap.")

130  have a full overlap.


In [670]:
notfullmatch=route_a_b[~route_a_b['Fullmatch']]
print(notfullmatch['route'].nunique()," do not have a full overlap.")

489  do not have a full overlap.


### Partial Overlap

In [826]:
route_a_b['intersec'] = route_a_b.apply(lambda x: set(x['asn']).intersection(set(x['origin'])), axis=1)
route_a_b['partial_match']=(route_a_b['intersec']!=set())
route_a_b['all_ases']=route_a_b.apply(lambda x: (x['asn'] | x['origin']), axis=1)

In [827]:
partial=route_a_b[(~route_a_b['Fullmatch']) & (route_a_b['partial_match'])]

In [837]:
print(partial['route'].nunique(),"prefixes have a partial overlap.")

177 prefixes have a partial overlap.


#### Do they belong to same org?

In [829]:
pa_mis_rs=partial['route'].unique() # checking that the AS partial mismatches might be from the same org
r2orgspa={}
ctr=0
tot=len(pa_mis_rs)
for r in tqdm(pa_mis_rs):
    r2orgspa[r]=[]
    ases_to_check=list(partial[partial['route']==str(r)]['all_ases'])[0]
    clean=[i.replace("'","") for i in ases_to_check]
    clean=[i.replace("AS","") for i in ases_to_check]
    for a in clean:
        if a in orgs_ases_gt:
            check_org=list(orgs[orgs['aut']==a]['org_id'].unique())[0]
            r2orgspa[r].append(check_org)
        if a not in orgs_ases_gt:
            r2orgspa[r].append('missing')
for r in r2orgspa:
    r2orgspa[r]=set(r2orgspa[r])

100%|██████████| 177/177 [00:06<00:00, 25.86it/s]


In [830]:
as_mismatch_same_org=[]
for r in r2orgspa:
    if (len(r2orgspa[r])==1) and (r2orgspa[r]!='missing'):
        as_mismatch_same_org.append(r)

In [831]:
print(len(as_mismatch_same_org),"prefixes belong to the same org.")

0 prefixes belong to the same org.


#### If not, is there a customer or peer relationship?

In [832]:
check_rel=partial[~partial['related']]['route'].unique()
related=[]
unrelated=[]
for r in tqdm(check_rel):
    check_ases=list(list(route_a_b[route_a_b['route']==str(r)]['all_ases'])[0])
    #use first AS to check relationships
    token_as=check_ases[0].lower().split('as')[1]
    #check if the rest of the ases are peers, customers, or providers
    token_peers=[]
    token_providers=[]
    token_customers=[]
    if token_as in myproviders:
        token_providers=myproviders[token_as]
    if token_as in mycustomers:
        token_customers=mycustomers[token_as]
    if token_as in mypeers:
        token_peers=mypeers[token_as]
    for a in check_ases:
        clean=a.lower().split('as')[1]
        if clean==token_as:
            continue
        if clean in token_peers:
            related.append(r)
        if clean in token_customers:
            related.append(r)
        if clean in token_providers:
            related.append(r)
    if r not in related:
        unrelated.append(r)
related=set(related)
unrelated=set(unrelated)

100%|██████████| 177/177 [00:00<00:00, 1516.30it/s]


In [833]:
len(related)+len(unrelated)

177

In [834]:
for i in list(route_a_b.index[route_a_b['route'].isin(related)]):
    route_a_b.at[i,'related']=True

In [835]:
print(len(related),"routes have a customer or provider or peer relationship.")

26 routes have a customer or provider or peer relationship.


In [836]:
print(len(unrelated),"routes do not have a customer or provider or peer relationship.")

151 routes do not have a customer or provider or peer relationship.


### No Overlap

In [673]:
print(route_a_b[~route_a_b['partial_match']]['route'].nunique(),"total mismatches.")

312 total mismatches.


In [674]:
total=route_a_b[~route_a_b['partial_match']]

In [648]:
tot_mis_rs=total['route'].unique() # checking that the AS mismatches might be from the same org
r2orgs={}
ctr=0
tot=len(tot_mis_rs)
for r in tqdm(tot_mis_rs):
    r2orgs[r]=[]
    ases_to_check=list(total[total['route']==str(r)]['all_ases'])[0]
    clean=[i.replace("'","") for i in ases_to_check]
    clean=[i.replace("AS","") for i in ases_to_check]
    for a in clean:
        if a in orgs_ases_gt:
            check_org=list(orgs[orgs['aut']==a]['org_id'].unique())[0]
            r2orgs[r].append(check_org)
        if a not in orgs_ases_gt:
            r2orgs[r].append('missing')
for r in r2orgs:
    r2orgs[r]=set(r2orgs[r])

100%|██████████| 312/312 [00:14<00:00, 21.74it/s]


In [649]:
as_mismatch_same_org=[]
for r in r2orgs:
    if (len(r2orgs[r])==1) and (r2orgs[r]!='missing'):
        as_mismatch_same_org.append(r)

In [650]:
print(len(as_mismatch_same_org),"prefixes belong to the same org.")

1 prefixes belong to the same org.


In [824]:
print(len(tot_mis_rs)-len(as_mismatch_same_org),"prefixes do not belong to the same org.")

311 prefixes do not belong to the same org.


In [671]:
route_a_b['same_org']=False
for i in list(route_a_b.index[route_a_b['route'].isin(as_mismatch_same_org)]):
    route_a_b.at[i,'same_org']=True
    route_a_b.at[i,'related']=True

#### If not, is there a customer or peer relationship?

In [815]:
check_rel=total[~total['related']]['route'].unique()
related=[]
unrelated=[]
for r in tqdm(check_rel):
    check_ases=list(list(route_a_b[route_a_b['route']==str(r)]['all_ases'])[0])
    #use first AS to check relationships
    token_as=check_ases[0].split('AS')[1]
    #check if the rest of the ases are peers, customers, or providers
    token_peers=[]
    token_providers=[]
    token_customers=[]
    if token_as in myproviders:
        token_providers=myproviders[token_as]
    if token_as in mycustomers:
        token_customers=mycustomers[token_as]
    if token_as in mypeers:
        token_peers=mypeers[token_as]
    for a in check_ases:
        clean=a.split('AS')[1]
        if clean==token_as:
            continue
        if clean in token_peers:
            related.append(r)
        if clean in token_customers:
            related.append(r)
        if clean in token_providers:
            related.append(r)
    if r not in related:
        unrelated.append(r)
related=set(related)
unrelated=set(unrelated)

100%|██████████| 311/311 [00:00<00:00, 1469.77it/s]


In [816]:
len(related)+len(unrelated)

311

In [820]:
for i in list(route_a_b.index[route_a_b['route'].isin(related)]):
    route_a_b.at[i,'related']=True

In [822]:
print(len(related),"routes have a customer or provider or peer relationship.")

16 routes have a customer or provider or peer relationship.


In [823]:
print(len(unrelated),"routes do not have a customer or provider or peer relationship.")

295 routes do not have a customer or provider or peer relationship.
